# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
! pip install ultralytics
! pip install torch torchvision torchaudio
! pip install  opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 980.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 11.5 MB/s eta 0:00:00


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [5]:
import cv2
from ultralytics import YOLO

model = YOLO('/content/YOLO_Model.pt')

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [17]:
#print(model)llمالها فايدة  بس اشيك على المودل

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 80, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(400, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_r

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [18]:
video_path = '/content/YOLO_Video.mp4'

In [23]:
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [24]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

In [25]:
print(f'Video width : {width} pixels')

Video width : 1920 pixels


In [26]:
heigh = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [27]:
print(f'Video heigh {heigh} pixels')

Video heigh 1080 pixels


In [28]:
fps = cap.get(cv2.CAP_PROP_FPS)

In [29]:
print(f' frames per second fbs {fps}')

 frames per second fbs 29.97002997002997


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [87]:
#output_path = '/content/YOLO_Video.mp4' لا تحطها في نفس الفايل
output_path = '/content/output_video_with_predictions.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc,29.97002997002997 , (1920, 1080))

In [86]:
fourcc =cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path,fourcc,fps,(width,heigh))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [88]:
# define the rol
rois = [
    (50,100,400,300),
    (450,100,800,300)

]
while cap.isOpened():
  ret,feame = cap.read()
  if not ret:
    break
    # drow rol
    for (X1,y1,X2,y2) in rois:
      cv2.rectangle(frame, (X1,y1),(X2,y2),(0,255,0),2)
      # display
      cv2.imshow('ROIs',frame)

      if cv2.waitKey(1) & 0xFF == ord('q'):
       break
      cap.release()
      cv2.destroyAllWindows


## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [89]:
import numpy as np

In [90]:
pip install opencv-python numpy

In [91]:
model = YOLO('/content/YOLO_Model.pt')

cap = cv2.VideoCapture('/content/output_video_with_predictions.mp4')

rois = [
    (50, 100, 400, 300),
    (450, 100, 800, 300)
]

exit_counts = {'top': 0, 'bottom': 0, 'left': 0, 'right': 0}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    boxes, confidences, class_ids = [], [], [] # المصدر>>(https://www.youtube.com/watch?v=8Jq3h2a-0yA)
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')
                x1, y1 = int(center_x - w / 2), int(center_y - h / 2)
                boxes.append([x1, y1, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in indices:
        i = i[0]
        box = boxes[i]
        (x, y, w, h) = box
        center_x = x + w // 2
        center_y = y + h // 2

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {class_ids[i]} | Conf: {confidences[i]:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
        if center_y < 100:
            exit_counts['top'] += 1
        elif center_y > height - 100:
            exit_counts['bottom'] += 1
        elif center_x < 100:
            exit_counts['left'] += 1
        elif center_x > width - 100:
            exit_counts['right'] += 1

    for (x1, y1, x2, y2) in rois:
         cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
         cv2.putText(frame, f'Exits - Top: {exit_counts["top"]}, Bottom: {exit_counts["bottom"]}, Left: {exit_counts["left"]}, Right: {exit_counts["right"]}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    cv2.putText(frame, 'car', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow('Vehicle Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



In [92]:
from google.colab import files

files.download('/content/output_video_with_predictions.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [82]:
 ! pip install -q mediapy

In [93]:
import mediapy as media
faile =('/content/output_video_with_predictions.mp4')

## Save and Submit Your Work

Submit both the notebook and the output video